In [9]:
from sim.simulator import *
from sim.sim_controllers import *
from sim.config import ExperimentConfig
from sim.motor_controllers import *
from utils.path_utils import join_paths
from utils.gui_utils import UserPrompt

# Configure the Simulation Parameters

In [4]:
################################ User Input ################################

# the input folder containing the logs of the experiment, i.e. the original bboxes.csv file and the experiment_config.json file
# if None, the user will be prompted to select the directory
input_folder = None

# the path to save the output logs of the selected controller
# if None, the user will be prompted to select the directory
output_folder = "logs"

############################################################################

if input_folder is None:
    input_folder = UserPrompt.open_directory("Select input directory containing the original logs of the experiment")

if output_folder is None:
    output_folder = UserPrompt.open_directory("Select output directory to save the logs of the selected controller")

# The oath pointing to the original bboxes.csv log file
input_log_path = join_paths(input_folder, "bboxes2.csv")

print(f"input folder: {input_folder}")
print(f"output folder: {output_folder}")

In [6]:
# Load the experiment config
experiment_config = ExperimentConfig.load_json(f"{input_folder}/experiment_config.json")

# initialize the logging configuration
# note that we do not save any images, since the simulator does not actually read
# any images from the disk, instead it reads the bboxes.csv file row by row. That allows for a major speedup in the simulation process.
# ONLY worth changing if you want to visualize the simulation, and if the simulator is provided with the original images.
log_config = LogConfig(
    root_folder=output_folder,
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=False,
)

################################ User Input ################################

# initialize the timing configuration
# changing this config changes the simulation settings, should remain the same 
# between different experiments
time_config = TimingConfig(
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.32, 0.32),
    experiment_config=experiment_config,
)

############################################################################

# Choose a Controller
Run the cell of the controller you need

**MLP Controller**  
Neural Network based location predictor.

In [13]:
import torch

################################ User Input ################################

# path to the MLP model file.
# if None, the user will be prompted to select the file
model_path = None  

############################################################################

if model_path is None:
    model_path = UserPrompt.open_file("Select the model file")

# load the model and create the controller
model: WormPredictor = torch.load(model_path)
controller = MLPController(time_config, input_log_path, model, model.io_config)

**Polyfit Controller**  
Weighted polynomial fitting location predictor.
The motion of the worm is fitted with a polynomial, which is afterwards used to predict the future location of the worm.

In [ ]:
## load configuration file and create the controller
poly_config = PolyfitConfig.load_json()
controller = PolyfitController(time_config, input_log_path)

**Optimal Controller**  
Approximation of the absolute best controller possible.

In [ ]:
controller = OptimalController(time_config, input_log_path)

**CSV Controller**  
Baseline controller without any future predictions.

In [ ]:
controller = CsvController(time_config, input_log_path)

**If you want to log the Simulation run the cell below as well**

In [ ]:
controller = LoggingController(controller, log_config)

# Run The Simulation

In [ ]:
# create motor controller which controls the motion of the platform 
motor_controller = SineMotorController(time_config)

# create the simulator
sim = Simulator(
    time_config,
    experiment_config,
    controller,
    reader=None,
    motor_controller=motor_controller,
)

In [ ]:
# run simulation
sim.run(visualize=False, wait_key=False)

In [ ]:
# save config files
experiment_config.save_json(join_paths(output_folder, "experiment_config.json"))
log_config.save_json(join_paths(output_folder, "log_config.json"))
time_config.save_json(join_paths(output_folder, "time_config.json"))

# Show the Results

In [ ]:
from eval.analysis import Plotter

pltr = Plotter(log_config.bbox_file_path, time_config)
data = pltr.print_statistics(n=10)

In [ ]:
pltr.plot_area_vs_speed()

In [ ]:
_ = pltr.plot_deviation()